# installing tensorflow and sorting images 

In [2]:
!pip install tensorflow


In [3]:
import tensorflow as tf

In [4]:
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.20.0


In [5]:
import tensorflow as tf

a = tf.constant(2)
b = tf.constant(3)
c = tf.add(a, b)

print(c.numpy())

5


In [6]:
import os
import time
import requests
import pandas as pd
from tqdm import tqdm  #adds a progress bar 
from urllib.parse import urlparse #helps extract the file extension (.jpg, .png, etc.) from a URL.
from os.path import splitext, basename #separate filename and extension from a URL path.
from IPython.display import Image, display #preview images inline.

# === 1) Load your CSV (same folder as the notebook) ===
CSV_NAME = "spurge-nettle.csv"  # <-- change if your filename differs
df = pd.read_csv(CSV_NAME)

# Quick peek so you can confirm column names
print(df.head(3))
print("Columns:", list(df.columns))

# === 2) Pick the image URL column automatically ===
# Add/adjust candidates if your CSV uses a different name
CANDIDATE_COLS = ["image_url", "photo_url", "url", "photo", "image"]
url_col = next((c for c in CANDIDATE_COLS if c in df.columns), None)
if url_col is None:
    raise ValueError(
        f"Could not find an image URL column. Look for one like {CANDIDATE_COLS} and set url_col manually."
    )

# Optional: prefer a stable identifier for filenames if present
id_col = next((c for c in ["id", "observation_id", "photo_id"] if c in df.columns), None)

# Unique, non-null URLs
image_urls = df[url_col].dropna().astype(str).unique()

# === 3) Where to save ===
OUT_DIR = "inat_images_spurge_nettle"
os.makedirs(OUT_DIR, exist_ok=True)

# === 4) Preview a few images inline (no download yet) ===
for url in image_urls[:5]:
    try:
        display(Image(url=url))
    except Exception as e:
        print(f"Preview failed for {url}: {e}")

# === 5) Helper to pick a reasonable filename ===
def filename_for(idx, url, row_id=None):
    # Try to preserve extension from URL; default to .jpg
    path = urlparse(url).path
    ext = splitext(path)[1].lower()
    if ext not in [".jpg", ".jpeg", ".png", ".gif", ".webp"]:
        ext = ".jpg"
    # Prefer an ID from the CSV when available
    if row_id is not None:
        return os.path.join(OUT_DIR, f"{row_id}{ext}")
    return os.path.join(OUT_DIR, f"photo_{idx}{ext}")

# Build a fast lookup from URL -> row id (if available)
id_by_url = {}
if id_col is not None:
    # Use first occurrence per URL
    first_indices = ~df[url_col].isna()
    for _, r in df.loc[first_indices, [url_col, id_col]].dropna(subset=[url_col]).iterrows():
        id_by_url.setdefault(str(r[url_col]), r[id_col])

# === 6) Download (resumable + polite) ===
headers = {"User-Agent": "iNat downloader for research (requests)"}  # polite header
errors = 0

for i, url in enumerate(tqdm(image_urls, desc="Downloading images")):
    row_id = id_by_url.get(url)
    out_path = filename_for(i, url, row_id=row_id)

    # Skip files already downloaded
    if os.path.exists(out_path):
        continue

    try:
        resp = requests.get(url, headers=headers, timeout=15)
        if resp.status_code == 200:
            with open(out_path, "wb") as f:
                f.write(resp.content)
        else:
            print(f"Failed {url} (HTTP {resp.status_code})")
            errors += 1
    except Exception as e:
        print(f"Error for {url}: {e}")
        errors += 1
        time.sleep(1)  # brief pause on error

    # Be gentle: tiny pause every 200 downloads
    if (i + 1) % 200 == 0:
        time.sleep(0.5)

print(f"Done. Saved images in: {OUT_DIR}. Errors: {errors}")


      id                                  uuid    observed_on_string  \
0  18042  b129acf2-22b7-49f1-a7d8-c9a8888b550e    May 07, 2009 09:21   
1  68317  528d10ac-74e1-4045-ac23-4d51802e5d47    May 11, 2011 11:25   
2  68318  0b311f52-e245-4997-8b52-4859c70a0101  March 26, 2012 12:16   

  observed_on         time_observed_at                   time_zone  user_id  \
0  2009-05-07  2009-05-07 16:21:00 UTC  Pacific Time (US & Canada)      604   
1  2011-05-11  2011-05-11 15:25:00 UTC  Eastern Time (US & Canada)     5464   
2  2012-03-26  2012-03-26 16:16:00 UTC  Eastern Time (US & Canada)     5464   

         user_login  user_name               created_at  ... geoprivacy  \
0         eric_hunt  Eric Hunt  2011-05-24 23:27:18 UTC  ...        NaN   
1  guerillafarmer99   SharpJ99  2012-04-21 00:00:23 UTC  ...        NaN   
2  guerillafarmer99   SharpJ99  2012-04-21 00:02:59 UTC  ...        NaN   

  taxon_geoprivacy coordinates_obscured positioning_method positioning_device  \
0           

Error for https://inaturalist-open-data.s3.amazonaws.com/photos/523401569/medium.jpg: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))


Done. Saved images in: inat_images_spurge_nettle. Errors: 7


In [6]:
!pip install pandas


In [4]:
!where python

C:\Users\cungv\anaconda3\python.exe
C:\Users\cungv\AppData\Local\Programs\Python\Python312\python.exe
C:\Users\cungv\AppData\Local\Microsoft\WindowsApps\python.exe


In [1]:
import os
import time
import requests
import pandas as pd
from tqdm import tqdm
from urllib.parse import urlparse
from os.path import splitext
from IPython.display import Image, display

# === 1) Load your CSV (make sure it's in the same folder as the notebook) ===
CSV_NAME = "texas-bull-nettle.csv"
df = pd.read_csv(CSV_NAME)

# Peek at the data
print(df.head(3))
print("Columns:", list(df.columns))

# === 2) Find the URL column automatically ===
CANDIDATE_COLS = ["image_url", "photo_url", "url", "photo", "image"]
url_col = next((c for c in CANDIDATE_COLS if c in df.columns), None)
if url_col is None:
    raise ValueError(f"No URL column found. Check columns: {list(df.columns)}")

# Optional: use an ID for filenames if available
id_col = next((c for c in ["id", "observation_id", "photo_id"] if c in df.columns), None)

# Unique, cleaned URLs
image_urls = df[url_col].dropna().astype(str).unique()

# === 3) Folder to save images ===
OUT_DIR = "inat_images_texas_bull_nettle"
os.makedirs(OUT_DIR, exist_ok=True)

# === 4) Preview a few images in Jupyter ===
for url in image_urls[:5]:
    try:
        display(Image(url=url))
    except Exception as e:
        print(f"Preview failed for {url}: {e}")

# === 5) Filename helper ===
def filename_for(idx, url, row_id=None):
    path = urlparse(url).path
    ext = splitext(path)[1].lower()
    if ext not in [".jpg", ".jpeg", ".png", ".gif", ".webp"]:
        ext = ".jpg"
    if row_id is not None:
        return os.path.join(OUT_DIR, f"{row_id}{ext}")
    return os.path.join(OUT_DIR, f"photo_{idx}{ext}")

# Map URL → ID (if available)
id_by_url = {}
if id_col is not None:
    for _, r in df[[url_col, id_col]].dropna(subset=[url_col]).iterrows():
        u = str(r[url_col])
        if u not in id_by_url:
            id_by_url[u] = r[id_col]

# === 6) Download with retry + polite pause ===
headers = {"User-Agent": "iNat downloader for research"}
errors = 0

for i, url in enumerate(tqdm(image_urls, desc="Downloading images")):
    row_id = id_by_url.get(url)
    out_path = filename_for(i, url, row_id=row_id)

    if os.path.exists(out_path):
        continue  # skip already downloaded

    try:
        resp = requests.get(url, headers=headers, timeout=15)
        if resp.status_code == 200:
            with open(out_path, "wb") as f:
                f.write(resp.content)
        else:
            print(f"Failed {url} (HTTP {resp.status_code})")
            errors += 1
    except Exception as e:
        print(f"Error for {url}: {e}")
        errors += 1
        time.sleep(1)

    if (i + 1) % 200 == 0:
        time.sleep(0.5)

print(f"Done. Saved images in: {OUT_DIR}. Errors: {errors}")


       id                                  uuid  \
0   98836  216cfaf8-b8ac-41a8-b345-ed307645dce7   
1  232803  daa6a01f-7dfd-418b-ade7-e0c6bb95929c   
2  242021  063d7d74-53d6-4046-90e5-3ac25fed083c   

                        observed_on_string observed_on  \
0                      July 04, 2012 18:14  2012-07-04   
1  Tue Apr 09 2013 08:14:55 GMT-0500 (CDT)  2013-04-09   
2  Sun Apr 21 2013 11:41:17 GMT-0500 (CDT)  2013-04-21   

          time_observed_at                   time_zone  user_id  user_login  \
0  2012-07-04 23:14:00 UTC  Central Time (US & Canada)     5429  gaberlunzi   
1  2013-04-09 13:14:55 UTC  Central Time (US & Canada)     9685     mikaelb   
2  2013-04-21 16:41:17 UTC  Central Time (US & Canada)     9685     mikaelb   

        user_name               created_at  ... geoprivacy taxon_geoprivacy  \
0             NaN  2012-07-05 14:30:26 UTC  ...        NaN              NaN   
1  Mikael Behrens  2013-04-09 15:04:35 UTC  ...        NaN              NaN   
2  Mikae

Done. Saved images in: inat_images_texas_bull_nettle. Errors: 0


In [2]:
import os
import pandas as pd

# ---- CONFIG: names & folders ----
DATA = [
    ("spurge-nettle.csv",        "inat_images_spurge_nettle"),
    ("texas-bull-nettle.csv",    "inat_images_texas_bull_nettle"),
]

CANDIDATE_URL_COLS = ["image_url", "photo_url", "url", "photo", "image"]
MIN_BYTES_TINY = 1024  # under 1KB likely broken

def find_url_col(df):
    for c in CANDIDATE_URL_COLS:
        if c in df.columns:
            return c
    raise ValueError(f"Could not find a URL column. Available columns: {list(df.columns)}")

def count_files(folder):
    if not os.path.exists(folder):
        return 0
    return sum(
        1 for f in os.listdir(folder)
        if os.path.isfile(os.path.join(folder, f))
    )

def tiny_files(folder, min_bytes=MIN_BYTES_TINY):
    if not os.path.exists(folder):
        return []
    tiny = []
    for f in os.listdir(folder):
        p = os.path.join(folder, f)
        if os.path.isfile(p):
            try:
                if os.path.getsize(p) < min_bytes:
                    tiny.append(f)
            except OSError:
                tiny.append(f"[stat_error]{f}")
    return tiny

summary = []

for csv_name, folder in DATA:
    # Load CSV
    df = pd.read_csv(csv_name)
    url_col = find_url_col(df)

    # Expected = unique, non-null URLs
    urls = df[url_col].dropna().astype(str).unique()
    expected = len(urls)

    # Downloaded = number of files present
    downloaded = count_files(folder)

    # Suspicious tiny files
    tiny = tiny_files(folder)
    tiny_count = len(tiny)

    # Quick status
    summary.append({
        "csv": csv_name,
        "folder": folder,
        "url_column": url_col,
        "expected_unique_urls": expected,
        "downloaded_files": downloaded,
        "missing_or_extra": expected - downloaded,  # positive => missing; negative => extras in folder
        "tiny_suspicious_files": tiny_count
    })

# Print the summary
for s in summary:
    print(f"=== {s['csv']} → {s['folder']} ===")
    print(f"URL column:              {s['url_column']}")
    print(f"Expected (unique URLs):  {s['expected_unique_urls']}")
    print(f"Downloaded (files):      {s['downloaded_files']}")
    miss_extra = s['missing_or_extra']
    if miss_extra > 0:
        print(f"Missing files:           {miss_extra}")
    elif miss_extra < 0:
        print(f"Extra files in folder:   {-miss_extra}")
    else:
        print("Counts match ✔")
    print(f"Tiny/suspicious files:   {s['tiny_suspicious_files']}")
    print()

# (Optional) show a few examples of tiny files for the first dataset if any
for csv_name, folder in DATA:
    t = tiny_files(folder)
    if t:
        print(f"Examples of tiny files in {folder}: {t[:10]}")
    else:
        print(f"No tiny files detected in {folder}.")


=== spurge-nettle.csv → inat_images_spurge_nettle ===
URL column:              image_url
Expected (unique URLs):  11537
Downloaded (files):      11530
Missing files:           7
Tiny/suspicious files:   0

=== texas-bull-nettle.csv → inat_images_texas_bull_nettle ===
URL column:              image_url
Expected (unique URLs):  10006
Downloaded (files):      10006
Counts match ✔
Tiny/suspicious files:   0

No tiny files detected in inat_images_spurge_nettle.
No tiny files detected in inat_images_texas_bull_nettle.


In [2]:
!pip install opencv-python


   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   ---- ----------------------------------- 4.2/39.0 MB 24.3 MB/s eta 0:00:02
   ------------- -------------------------- 12.8/39.0 MB 31.8 MB/s eta 0:00:01
   -------------------- ------------------- 20.2/39.0 MB 35.9 MB/s eta 0:00:01
   ------------------------------ --------- 29.4/39.0 MB 34.1 MB/s eta 0:00:01
   -------------------------------------- - 37.2/39.0 MB 34.8 MB/s eta 0:00:01
   ---------------------------------------  38.8/39.0 MB 35.0 MB/s eta 0:00:01
   ---------------------------------------- 39.0/39.0 MB 29.3 MB/s  0:00:01


# labeling flowering 

In [1]:
# %pip install ipywidgets   # ← run once if needed, then restart the kernel

import os, glob, shutil
from IPython.display import display, clear_output
import ipywidgets as widgets
from PIL import Image
import matplotlib.pyplot as plt

# ----- CONFIG -----
SOURCE = "random_1000_flowers"     # folder with unsorted images
OUTROOT = "1000_labeled_flowers"   # destination root
FLOWER_DIR = os.path.join(OUTROOT, "flower")
NON_DIR    = os.path.join(OUTROOT, "non_flower")

os.makedirs(FLOWER_DIR, exist_ok=True)
os.makedirs(NON_DIR, exist_ok=True)

# Collect remaining images to label (skip hidden + non-images)
queue = [p for p in glob.glob(os.path.join(SOURCE, "*"))
         if p.lower().endswith((".jpg",".jpeg",".png")) and os.path.isfile(p)]
idx = 0

# Widgets
img_out = widgets.Output()
status  = widgets.HTML()
btn_flower = widgets.Button(description=" Flower", button_style="success")
btn_non    = widgets.Button(description=" Non-Flower", button_style="danger")
btn_skip   = widgets.Button(description=" Skip")
btn_back   = widgets.Button(description=" Back")

def show():
    img_out.clear_output(wait=True)
    remaining = len(queue) - idx
    f_count = len([f for f in os.listdir(FLOWER_DIR) if f.lower().endswith(('.jpg','.jpeg','.png'))])
    n_count = len([f for f in os.listdir(NON_DIR) if f.lower().endswith(('.jpg','.jpeg','.png'))])
    if idx >= len(queue):
        status.value = f"✅ Done. Counts — flower: {f_count}, non_flower: {n_count}"
        with img_out: print("No more images to label.")
        return
    path = queue[idx]
    status.value = f"{idx+1}/{len(queue)}  |  remaining: {remaining}  |  flower: {f_count}  non_flower: {n_count}"
    with img_out:
        clear_output(wait=True)
        plt.figure(figsize=(6,6))
        try:
            im = Image.open(path).convert("RGB")
        except Exception as e:
            print(f"⚠️ Unable to open {os.path.basename(path)}: {e}")
            return
        plt.imshow(im); plt.axis("off")
        plt.title(os.path.basename(path))
        plt.show()

def unique_dest(dst_dir, src_path):
    base = os.path.join(dst_dir, os.path.basename(src_path))
    root, ext = os.path.splitext(base)
    k, dst = 1, base
    while os.path.exists(dst):
        dst = f"{root}__{k}{ext}"
        k += 1
    return dst

def move_to(dst_dir):
    global idx
    if idx >= len(queue): return
    src = queue[idx]
    dst = unique_dest(dst_dir, src)
    shutil.move(src, dst)
    idx += 1
    show()

def skip(_):
    global idx
    # push current item to end of queue
    if idx < len(queue):
        queue.append(queue.pop(idx))
    show()

def back(_):
    global idx
    idx = max(0, idx-1)
    show()

btn_flower.on_click(lambda _: move_to(FLOWER_DIR))
btn_non.on_click(lambda _: move_to(NON_DIR))
btn_skip.on_click(skip)
btn_back.on_click(back)

display(widgets.HBox([btn_back, btn_skip, btn_flower, btn_non]), status, img_out)
show()


HTML(value='')

Output()

# downloading random images 

In [2]:
import os
import random
import shutil

# --- 1. Configure your folders ---
source_folder1 = "inat_images_texas_bull_nettle"   # First source folder
source_folder2 = "inat_images_spurge_nettle"       # Second source folder
destination_folder = "random_300_flowers"          # Destination folder
num_images_per_folder = 150                        # How many to copy from each

# --- 2. Make sure destination folder exists ---
os.makedirs(destination_folder, exist_ok=True)

# --- 3. Helper function to randomly copy files ---
def copy_random_images(source, dest, n):
    all_images = [f for f in os.listdir(source)
                  if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    if not all_images:
        print(f"⚠️ No images found in {source}")
        return
    
    selected = random.sample(all_images, min(n, len(all_images)))
    for img in selected:
        src_path = os.path.join(source, img)
        dst_path = os.path.join(dest, f"{os.path.basename(source)}_{img}")  # avoid overwriting
        shutil.copy(src_path, dst_path)
    print(f"✅ Copied {len(selected)} images from {source}")

# --- 4. Copy from both folders ---
copy_random_images(source_folder1, destination_folder, num_images_per_folder)
copy_random_images(source_folder2, destination_folder, num_images_per_folder)

print(f"\n📸 Total images in {destination_folder}: {len(os.listdir(destination_folder))}")


✅ Copied 150 images from inat_images_texas_bull_nettle
✅ Copied 150 images from inat_images_spurge_nettle

📸 Total images in random_300_flowers: 300
